<a href="https://colab.research.google.com/github/RodrigoRG24/Fake-Waze/blob/main/Fake_Waze_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install perlin_noise
import matplotlib.pyplot as plt
import numpy as np
from perlin_noise import PerlinNoise
# ------------
import csv
import json
# Sección Aristas
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
import random
# Dijkstra/Priority Queue
import heapq as hq
import math
# ------------
import numpy as np
import graphviz as gv #sirve para dibujar grafos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def addKey_Values(dictionary, key, value):
    dictionary[key] = value

In [ ]:
def printDictionary(dictionary):
    for k, v in dictionary.items():
        print(f"{k}: {v}")

In [ ]:
import csv
def cargarDatos(dictionary):
    with open('Calles de Lima.csv') as file:
        reader = csv.reader(file)

        for id, row in enumerate(reader):
            addKey_Values(dictionary, row[0].lower(), id)

In [ ]:
def createStreetsDictionary():
    streets = {}
    cargarDatos(streets)
    return streets

In [ ]:
streetsDictionary = createStreetsDictionary()
printDictionary(streetsDictionary)

prolongacion naranjal: 0
entrada al templo huaca el paraiso: 1
calle uranio: 2
el sol de naranjal: 3
avenida josefina: 4
avenida naranjal: 5
calle e: 6
calle 28 de mayo: 7
calle los petalos: 8
avenida paramonga: 9
santa cruz: 10
santa rosa: 11
calle las violetas: 12
virgen del rosario: 13
calle llata: 14
san nicolas: 15
los eucaliptos: 16
los claveles: 17
calle las palmas: 18
las gardenias: 19
circa patria: 20
avenida central: 21
avenida pacasmayo: 22
los geranios 2: 23
jiron las americas: 24
miguel grau: 25
avenida san jose: 26
los portales de chavin: 27
calle los claveles: 28
calle los ficus: 29
avenida canta callao: 30
los nogales: 31
el olivar: 32
calle bolognesi: 33
avenida los alisos: 34
avenida santa rosa: 35
san pablo: 36
avenida antigua pacasmayo: 37
avenida carlos izaguirre: 38
calle jose olaya: 39
calle maria parado de bellido: 40
san agustin: 41
los robles: 42
calle progreso: 43
los canamos: 44
calle las esmeraldas: 45
jiron los pinares: 46
calle los nogales: 47
jiron los r

In [ ]:
def printAdList(graph):
    for i, intersection in enumerate(graph):
        print(i, intersection)

    print(f"Cantidad de vértices = {len(graph)}")

In [ ]:
def streetExists(streetName):
    if streetName in streetsDictionary:
        id = streetsDictionary[streetName]
    else:
        id = -1 # la calle no está mapeada
    return id

In [ ]:
def createAdjacencyList(adjacencyList):
    with open('Calles de Lima.csv') as file:
        reader = csv.reader(file)
        for row in reader:
            intersections = []
            for i in range(len(row)):
                if i != 0:
                    intersections.append(streetExists(row[i].lower()))
            adjacencyList.append(intersections)

    return adjacencyList

In [ ]:
intersectionsAdList = createAdjacencyList([])
printAdList(intersectionsAdList)

0 [1, 2, 3, 4]
1 [0]
2 [0]
3 [0, 4, 15, 16, 9, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 29, 28, 5, 30, 25]
4 [9, 3, 5, 0, 84]
5 [4, 15, 6, 2, 9, 17, 19, 21, 7, 22, -1, 8, 24, 10, 11, 12, 13, -1, 14, -1, 30, 58]
6 [5, -1, -1]
7 [5, 110, -1, -1]
8 [5]
9 [4, 15, 3, 16, 32]
10 [5]
11 [5]
12 [5]
13 [5]
14 [5, -1]
15 [3, 9, 31, 32, 19, 51, 54, 34]
16 [3, 19, 32, 31, 9]
17 [3]
18 [3]
19 [3]
20 [3]
21 [3, 32]
22 [3, 32, 33, 34, 35, 36, 37, 38, 105]
23 [3, 39, 40]
24 [3, -1]
25 [3]
26 [3, 41, 42, 43, 44, 45, 46, 34]
27 [3]
28 [3]
29 [3, 47, 48]
30 [3, 58, 61, 34, 62, 38, 63, 22, 64, 88]
31 [15, 49, 16]
32 [15, 49, 16, 9]
33 [22, 50]
34 [58, 30, 65, 62, 26, 68, 22, 69, 70, 71]
35 [22, 69, 72, 73, 70, 37, 76]
36 [22, 72, 77, 37, 79, 80]
37 [35, 81, 36, 22]
38 [64, 22, 58, 30]
39 [23, 50]
40 [23]
41 [26, 52]
42 [26]
43 [26, 52]
44 [-1]
45 [26, -1]
46 [26, 65]
47 [29, 55, 56, 57, 58]
48 [29, 82]
49 [31, -1, 32]
50 [39, 107, 33]
51 [52, 53, -1, 15]
52 [51]
53 [51, 83, 54]
54 [53, -1, 15]
55 [47]
56 [

In [ ]:
def createKeys(graph):
    intersectionsIDs = {}

    for i in range(len(graph)):
        for street in graph[i]:
            # si las keys aún no han sido creadas, las agrega al diccionario
            if (i, street) not in intersectionsIDs and street != -1:
                intersectionsIDs[(i, street)] = []

    return intersectionsIDs

In [ ]:
def addIntersections(graph, i, j):
    street = graph[i][j]

    if street != -1:
        # agrego la intersección már próxima hacia la izquierda
        if j > 0 and graph[i][j - 1] != -1:
            tup = (i, graph[i][j - 1])
            map[(i, street)].append(tup)

        # agrego la intersección már próxima hacia la derecha
        if j < len(graph[i]) - 1 and graph[i][j + 1] != -1:
            tup = (i, graph[i][j + 1])
            map[(i, street)].append(tup)

In [ ]:
def addIntersectionsReversed(graph, i, j):
    street = graph[i][j]

    # ahora voy al reverso de la lista
    if i != -1 and street != -1:
        if (street < len(graph)) and (i in graph[street]):
            tup = (street, i)
            map[(i, street)].append(tup)

            k = graph[street].index(i) # k = 2

            if k > 0 and graph[street][k - 1] != -1:
                tup = (street, graph[street][k - 1])
                map[(i, street)].append(tup)

            if k < len(graph[street]) - 1 and graph[street][k + 1] != -1:
                tup = (street, graph[street][k + 1])
                map[(i, street)].append(tup)

In [ ]:
def joinIntersections(graph):
    n = len(graph)
    for i in range(n):
        for j in range(len(graph[i])):
            addIntersections(graph, i, j)
            addIntersectionsReversed(graph, i, j)

In [ ]:
def totalNodes(graph):
    total = 0
    for key, value in graph.items():
        total += len(value)
    return total

In [ ]:
map = createKeys(intersectionsAdList)
joinIntersections(intersectionsAdList)
print(map)

{(0, 1): [(0, 2), (1, 0)], (0, 2): [(0, 1), (0, 3), (2, 0)], (0, 3): [(0, 2), (0, 4), (3, 0), (3, 4)], (0, 4): [(0, 3), (4, 0), (4, 5), (4, 84)], (1, 0): [(0, 1), (0, 2)], (2, 0): [(0, 2), (0, 1), (0, 3)], (3, 0): [(3, 4), (0, 3), (0, 2), (0, 4)], (3, 4): [(3, 0), (3, 15), (4, 3), (4, 9), (4, 5)], (3, 15): [(3, 4), (3, 16), (15, 3), (15, 9)], (3, 16): [(3, 15), (3, 9), (16, 3), (16, 19)], (3, 9): [(3, 16), (3, 17), (9, 3), (9, 15), (9, 16)], (3, 17): [(3, 9), (3, 18), (17, 3)], (3, 18): [(3, 17), (3, 19), (18, 3)], (3, 19): [(3, 18), (3, 20), (19, 3)], (3, 20): [(3, 19), (3, 21), (20, 3)], (3, 21): [(3, 20), (3, 22), (21, 3), (21, 32)], (3, 22): [(3, 21), (3, 23), (22, 3), (22, 32)], (3, 23): [(3, 22), (3, 24), (23, 3), (23, 39)], (3, 24): [(3, 23), (3, 26), (24, 3)], (3, 26): [(3, 24), (3, 27), (26, 3), (26, 41)], (3, 27): [(3, 26), (3, 29), (27, 3)], (3, 29): [(3, 27), (3, 28), (29, 3), (29, 47)], (3, 28): [(3, 29), (3, 5), (28, 3)], (3, 5): [(3, 28), (3, 30)], (3, 30): [(3, 5), (3, 

## **LATITUDES Y LONGITUDES**

In [ ]:
def formatearLatitudeorLongitude(input):
    separar = input
    separar = separar.split(".");

    union= separar[0]+"."+separar[1]+separar[2];
    variable_float = float(union)

    return variable_float

In [ ]:
def getOnlyLatLong(key):
    return (latlong[key][0], latlong[key][1])

def getInterId(key):
    return latlong[key][2]

In [ ]:
def createLatLongDictionary(latlong):
    with open("Latitud y Longitud de calles.csv") as file:
        reader = csv.reader(file)
        # ignora el header (la 1era fila del csv)
        next(reader)
        # se le asignará un id a cada vértice
        id = 0
        for row in reader:
            # busco el id de la calle en mi diccionario de calles
            intersection = streetExists(row[0]), streetExists(row[1])
            if intersection in map:
                latlong[intersection] = formatearLatitudeorLongitude(row[2]), formatearLatitudeorLongitude(row[3]), id
                id += 1
    return latlong

In [ ]:
latlong = createLatLongDictionary({})
print(latlong)
printDictionary(latlong)

{(0, 1): (-11.948377, -77.114573, 0), (0, 2): (-11.95225, -77.109731, 1), (0, 3): (-11.95115, -77.112078, 2), (0, 4): (-11.954132, -77.105738, 3), (1, 0): (-11.948377, -77.114573, 4), (2, 0): (-11.95225, -77.109731, 5), (3, 0): (-11.95115, -77.112078, 6), (3, 4): (-11.95931, -77.111275, 7), (3, 15): (-11.96261, -77.107892, 8), (3, 16): (-11.964076, -77.10626, 9), (3, 9): (-11.966049, -77.104222, 10), (3, 17): (-11.966339, -77.103778, 11), (3, 18): (-11.967033, -77.102723, 12), (3, 19): (-11.967239, -77.102303, 13), (3, 20): (-11.967534, -77.101308, 14), (3, 21): (-11.967833, -77.099961, 15), (3, 22): (-11.968343, -77.097, 16), (3, 23): (-11.970205, -77.09599, 17), (3, 24): (-11.970902, -77.095274, 18), (3, 26): (-11.971864, -77.094194, 19), (3, 27): (-11.972385, -77.093372, 20), (3, 29): (-11.973014, -77.091181, 21), (3, 28): (-11.972659, -77.09238, 22), (3, 5): (-11.973722, -77.088578, 23), (3, 30): (-11.973864, -77.088405, 24), (3, 25): (-11.971613, -77.094397, 25), (4, 9): (-11.9624

##**Agregar valores aleatorios**

In [ ]:
def getGraphSize(graph):
    return len(graph)

In [ ]:
def perlinNoise():
    # tamaño de cada de arreglo
    xpix = 10
    # cantidad de arreglos
    ypix = getGraphSize(map)
    noise = PerlinNoise(octaves=5, seed=1)
    pic = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]

    return np.array(np.array(pic)*100, dtype=int)

In [ ]:
randValues = perlinNoise()

In [ ]:
def sameNode(n1, n2):
    x1, y1 = n1
    x2, y2 = n2

    if (x1 == y2) and (y1 == x2):
        return True
    return False

In [ ]:
def hasAzero(list_randValues):
    if 0 in list_randValues:
        return True
    return False

In [ ]:
def getListRandValues(n):
    r = random.randrange(0, n)

    #while hasAzero(randValues[r]) == True:
    while sum(randValues[r]) == 0:
        r = random.randrange(0, n)

    return list(randValues[r])

In [ ]:
def getPositive(num):
    if num < 0:
        return num * -1
    else:
        return num

In [ ]:
def addRandValue(dictionary):

    randv = getListRandValues(len(latlong))

    for key, value in map.items():
        l = []
        for u in value:
            if sameNode(key, u):
                u += (0,)
                l.append(u)
                continue

            if len(randv) > 0:
                u += (getPositive(randv.pop()),)
            else:
                randv = getListRandValues(len(latlong))
                u += (getPositive(randv.pop()),)
            l.append(u)
        dictionary[key] = l
    return dictionary

In [ ]:
map2 = addRandValue({})
printDictionary(map2)

(0, 1): [(0, 2, 5), (1, 0, 0)]
(0, 2): [(0, 1, 4), (0, 3, 4), (2, 0, 0)]
(0, 3): [(0, 2, 3), (0, 4, 2), (3, 0, 0), (3, 4, 2)]
(0, 4): [(0, 3, 1), (4, 0, 0), (4, 5, 1), (4, 84, 0)]
(1, 0): [(0, 1, 0), (0, 2, 0)]
(2, 0): [(0, 2, 0), (0, 1, 1), (0, 3, 1)]
(3, 0): [(3, 4, 1), (0, 3, 0), (0, 2, 0), (0, 4, 0)]
(3, 4): [(3, 0, 0), (3, 15, 0), (4, 3, 0), (4, 9, 0), (4, 5, 0)]
(3, 15): [(3, 4, 0), (3, 16, 5), (15, 3, 0), (15, 9, 4)]
(3, 16): [(3, 15, 4), (3, 9, 3), (16, 3, 0), (16, 19, 2)]
(3, 9): [(3, 16, 2), (3, 17, 1), (9, 3, 0), (9, 15, 1), (9, 16, 0)]
(3, 17): [(3, 9, 0), (3, 18, 3), (17, 3, 0)]
(3, 18): [(3, 17, 3), (3, 19, 2), (18, 3, 0)]
(3, 19): [(3, 18, 2), (3, 20, 1), (19, 3, 0)]
(3, 20): [(3, 19, 1), (3, 21, 1), (20, 3, 0)]
(3, 21): [(3, 20, 0), (3, 22, 0), (21, 3, 0), (21, 32, 0)]
(3, 22): [(3, 21, 2), (3, 23, 2), (22, 3, 0), (22, 32, 2)]
(3, 23): [(3, 22, 2), (3, 24, 2), (23, 3, 0), (23, 39, 2)]
(3, 24): [(3, 23, 1), (3, 26, 1), (24, 3, 0)]
(3, 26): [(3, 24, 1), (3, 27, 1), (26, 3

##**Escribir archivos**

In [ ]:
def writeNodes():
    with open("nodes.txt", "w") as f:
        with open('Calles de Lima.csv') as file:
            reader = csv.reader(file)

            for row in reader:
                for i in range(1, len(row)):
                    f.write(f"{row[0]}, {row[i]}\n")
writeNodes()

In [ ]:
def writeStreetsDictionary():
    with open("streetNames.txt", "w") as f:
        for key, value in streetsDictionary.items():
            f.write(key + "\n")
writeStreetsDictionary()

In [ ]:
def writeStreetsAdList():
    with open("streetsAdList.txt", "w") as f:
        for streets in intersectionsAdList:
            for i in range(len(streets)):
                streets[i] = str(streets[i])
            f.write(', '.join(streets) + "\n")
writeStreetsAdList()

In [ ]:
def writeStreetMap():
    with open("map.txt", "w") as f:
        for key, value in map.items():
            listWOCommas = (str(value).strip('[]')).replace(",", "")
            f.write(str(key) + " " + listWOCommas + '\n')
writeStreetMap()

##**Aristas**

In [ ]:
def getCurrentTime():

    now = datetime.now()
    time = now.strftime("%H %p")
    time = time.split(" ")

    # convierto la hora a la timezone de Perú
    if time[0] <= "5":
        time[0] = 24 - (5 - int(time[0]))
    else:
        time[0] = int(time[0]) - 5

    return str(time[0])

def askTime():
    time = int(input("Digite la hora en formato de 24hrs: "));
    if time not in range(0, 24):
        time = askTime()
    return time

In [ ]:
def getlongitude(n1, n2):
    lon1,lat1=n1
    lon2,lat2=n2
    if lon1 == lat2 and lon2 == lat1:
      longitude = 0
    else:
      lon1 = radians(lon1)
      lon2 = radians(lon2)
      lat1 = radians(lat1)
      lat2 = radians(lat2)
      dlon = lon2 - lon1
      dlat = lat2 - lat1
      a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
      c = 2 * asin(sqrt(a))
      r = 6371
      longitude = c * r

    return longitude

In [ ]:
def point_percent(time):
    percent = 0
    # de 0am - 4am
    if time in range(0, 5):
        percent = 0.1

    # de 5am - 8am
    if time in range(5, 9):
        percent = 0.9

    # de 9am - 16pm
    if time in range(9, 18):
        percent = 0.5

    # de 18pm - 20pm
    if time in range(18, 21):
        percent = 1.3

    # de 21pm - 23pm
    if time in range(21, 23):
        percent = 0.3

    return percent

In [ ]:
def getFactor(time, val):
    # el factor está en el rango de 0 - 50
    # dependiendo de la hora, mi valor va estar en un determinado rango
    # puntos por hora
    # índice: hora, elemento: puntaje
                 # 0  1  2  3  4  5  6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23
    time_points = [0, 1, 1, 1, 2, 3, 15, 19, 17, 14, 13, 10, 10, 11, 12, 13, 14, 16, 17, 20, 19, 15, 4, 2]

    percent = point_percent(time)
    factor = int((val * percent) + time_points[time])

    if factor > 50:
        factor = 50

    return factor

In [ ]:
def calculateWeight(n1, n2, time, val):
    longitude = getlongitude(n1, n2) * 100
    factor = getFactor(time, val)

    weight = (longitude * factor)

    if longitude > 0:
        weight = round(factor + longitude, 2)

    if (weight > 100):
        return 99
    else:
        return int(weight)

In [ ]:
def updateGraph():
    time = askTime()

    # map con pesos
    mapW = {}
    for key in map2:
        if key in latlong:
            k = getInterId(key)
            mapW[k] = []
        else:
            continue
        for arista in map2[key]:
            edge = (arista[0], arista[1])
            if edge in latlong:
                weight = calculateWeight(getOnlyLatLong(key), getOnlyLatLong(edge), time, arista[2])
                # (intersección, peso)
                if weight == 0: continue
                mapW[k].append((getInterId(edge), weight))
        mapW[k]
    return mapW

In [ ]:
def getMaxKey(G):
    return max(G, key = G.get)

In [ ]:
mapW = updateGraph()

Digite la hora en formato de 24hrs: 10


##**Shortest Paths**

In [ ]:
def getMin(a):
    min = a[0][1]
    index = 0
    for i, c in enumerate(a):
        if c[1] < min:
            min = c[1]
            index = i
    return a.pop(index)

In [ ]:
def definePath(d, path):
    g = []
    g.append(d)

    while path[d] != -1:
        destination = path[d]
        g.insert(0, destination)
        d = destination
    return g

In [ ]:
def findShortestPath(graph, s, ignore):
    n = len(graph)
    path = [-1] * n
    visited = [False] * n
    visited[ignore] = True

    cost = [math.inf] * n
    cost[s] = 0

    # priority queue
    # vértice, peso
    priorityq = [(s, 0)]

    while priorityq:
        # vértice, peso
        u, g = getMin(priorityq)
        if not visited[u]:
            visited[u] = True
        for v, w in graph[u]:
            if not visited[v]:
                weight = g + w
                if weight < cost[v]:
                    cost[v] = weight
                    path[v] = u
                    priorityq.append((v, weight))
    return path, cost

In [ ]:
def getDirections(s, d, graph):
    neighbours = graph[s]
    paths = []
    costs = []

    for n in neighbours:
        path, cost = findShortestPath(mapW, n[0], s)
        path = definePath(d, path)
        path.insert(0, s)
        paths.append(path)
        costs.append(cost[d])
    return paths, costs

In [ ]:
def getBestPaths(paths, costs):
    bestCosts = [(i, x) for i, x in enumerate(costs)]
    bestCosts.sort(key=lambda x:x[1])

    n = len(bestCosts)
    if n > 3:
        bestCosts = [bestCosts[i] for i in range(3)]
    else:
        bestCosts = [bestCosts[i] for i in range(n)]

    bestPaths = [paths[c[0]] for c in bestCosts]

    return bestPaths

In [ ]:
def aPathExists(paths, costs):
    for i, cost in enumerate(costs):
        if cost == float('Inf'):
            paths[i] = []

In [ ]:
def deleteInvalidPaths(paths, costs):
    for i, p in enumerate(paths):
        if p == []:
            paths.pop(i)
            costs.pop(i)

In [ ]:
source = 10
# a donde quiero llegar
destination = 27
paths, costs = getDirections(source, destination, mapW)

# elimina caminos inválidos (no existentes)
aPathExists(paths, costs)
deleteInvalidPaths(paths, costs)

# retorna los mejores caminos
paths = getBestPaths(paths, costs)
costs.sort()

In [ ]:
print(paths, costs)

[[10, 55, 54, 27], [10, 9, 8, 65, 54, 27], [10, 57, 75, 121, 120, 65, 54, 27]] [103, 187, 233]


##**Visualizaciones**

In [ ]:
def latlongGraph(dictionary):
    n = len(dictionary) + 10
    latlongs = [()] * n

    for key, value in dictionary.items():
        latlongs[dictionary[key][2]] = dictionary[key][0] * -1, dictionary[key][1] * -1

    return latlongs

In [ ]:
printAdList(latlongGraph(latlong))

0 (11.948377, 77.114573)
1 (11.95225, 77.109731)
2 (11.95115, 77.112078)
3 (11.954132, 77.105738)
4 (11.948377, 77.114573)
5 (11.95225, 77.109731)
6 (11.95115, 77.112078)
7 (11.95931, 77.111275)
8 (11.96261, 77.107892)
9 (11.964076, 77.10626)
10 (11.966049, 77.104222)
11 (11.966339, 77.103778)
12 (11.967033, 77.102723)
13 (11.967239, 77.102303)
14 (11.967534, 77.101308)
15 (11.967833, 77.099961)
16 (11.968343, 77.097)
17 (11.970205, 77.09599)
18 (11.970902, 77.095274)
19 (11.971864, 77.094194)
20 (11.972385, 77.093372)
21 (11.973014, 77.091181)
22 (11.972659, 77.09238)
23 (11.973722, 77.088578)
24 (11.973864, 77.088405)
25 (11.971613, 77.094397)
26 (11.962428, 77.114461)
27 (11.95931, 77.111275)
28 (11.954392, 77.105983)
29 (11.954132, 77.105738)
30 (11.954023, 77.105622)
31 (11.954392, 77.105983)
32 (11.95751, 77.102505)
33 (11.958967, 77.101589)
34 (11.959443, 77.101175)
35 (11.961302, 77.099446)
36 (11.96168, 77.099103)
37 (11.963186, 77.097843)
38 (11.964011, 77.097114)
39 (11.9654

In [ ]:
def getGraph(graph):
    n = len(graph) + 10
    G = [[(510, 0)]] * n

    for key, value in graph.items():
        G[key] = value
    return G

In [ ]:
latlongs = latlongGraph(latlong); graph = getGraph(mapW)

In [ ]:
print(latlongs)
print(graph)

[(11.948377, 77.114573), (11.95225, 77.109731), (11.95115, 77.112078), (11.954132, 77.105738), (11.948377, 77.114573), (11.95225, 77.109731), (11.95115, 77.112078), (11.95931, 77.111275), (11.96261, 77.107892), (11.964076, 77.10626), (11.966049, 77.104222), (11.966339, 77.103778), (11.967033, 77.102723), (11.967239, 77.102303), (11.967534, 77.101308), (11.967833, 77.099961), (11.968343, 77.097), (11.970205, 77.09599), (11.970902, 77.095274), (11.971864, 77.094194), (11.972385, 77.093372), (11.973014, 77.091181), (11.972659, 77.09238), (11.973722, 77.088578), (11.973864, 77.088405), (11.971613, 77.094397), (11.962428, 77.114461), (11.95931, 77.111275), (11.954392, 77.105983), (11.954132, 77.105738), (11.954023, 77.105622), (11.954392, 77.105983), (11.95751, 77.102505), (11.958967, 77.101589), (11.959443, 77.101175), (11.961302, 77.099446), (11.96168, 77.099103), (11.963186, 77.097843), (11.964011, 77.097114), (11.965461, 77.095791), (11.966591, 77.094733), (11.968776, 77.092669), (11.96